In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from sqlalchemy import text


In [ ]:
load_dotenv()
password=os.getenv("password")


Establish a connection between Python and the Sakila database.

In [ ]:
bd = "Sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)
engine


Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

    engine: an object representing the database connection engine to be used to establish a connection to the  Sakila database.
    
    month: an integer representing the month for which rental data is to be retrieved.
    
    year: an integer representing the year for which rental data is to be retrieved.


In [ ]:
def rentals_month(engine,month,year):
    enter_month=month
    enter_year=year
    with engine.connect() as connection:
        table= text(f"select *, month(rental_date) as 'month', year(rental_date) as 'year' from rental where month(rental_date) = '{enter_month}' and year(rental_date)= '{enter_year}';") 
        info1=text("select month(rental_date) as 'month' from rental group by month(rental_date);")
        info2=text("select year(rental_date) as 'year' from rental group by year(rental_date);")
        info_1=connection.execute(info1)
        info_2=connection.execute(info2)
        data= connection.execute(table)
        print(f"These database counts with the months: {info_1.all()} and years: {info_2.all()} that are in the talbe")
        df=pd.DataFrame(data.all())
    return df

In [ ]:
data=rentals_month(engine,2,2006)
data


Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

In [ ]:
def rental_count_month(df_rentals_month,month,year):
    enter_month=month
    enter_year=year

    rentas_customer=pd.DataFrame(df_rentals_month.groupby('customer_id')[['inventory_id']].count())

    rentas_customer.rename(columns= {'inventory_id': f"rentals_{enter_month}_{enter_year}"}, inplace=True)
    
    return rentas_customer


In [ ]:
rental_count_month(data,2,2006)

Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [ ]:
def compare_rentals(df_one,df_two):
    combined_df=df_one.join([df_two],how='outer')
    combined_df.iloc[:,0].fillna(0, inplace=True)
    combined_df.iloc[:,1].fillna(0, inplace=True)
    combined_df["difference"]=combined_df.iloc[:,1]-combined_df.iloc[:,0]
    
    return combined_df

In [ ]:
info_one=rentals_month(engine,5,2005)
df_one=rental_count_month(info_one,5,2005)
info_two=rentals_month(engine,6,2005)
df_two=rental_count_month(info_two,6,2005)

compare_rentals(df_one,df_two)